# String to Number Conversion - to_num()

The `to_num()` function provides robust conversion of various input types to numeric values with validation and precision control.

**Core Features:**
- **Type Flexibility**: Handles strings, int, float, bool, and Decimal inputs
- **Type Targeting**: Convert to int or float with `num_type` parameter
- **Bounds Validation**: Enforce upper and lower bounds with clear error messages
- **Precision Control**: Round floats to specified decimal places
- **Robust Validation**: Comprehensive error handling for invalid inputs

In [1]:
from decimal import Decimal

from lionherd_core.libs.string_handlers import to_num

## 1. Basic Type Conversion

Convert strings, integers, and floats to numeric types. Default target is `float`.

In [2]:
# String to float (default)
result = to_num("42.5")
print(f"String '42.5' → {result} (type: {type(result).__name__})")

# Integer input
result_int = to_num(100)
print(f"Int 100 → {result_int} (type: {type(result_int).__name__})")

# Float input (passthrough)
result_float = to_num(3.14159)
print(f"Float 3.14159 → {result_float} (type: {type(result_float).__name__})")

String '42.5' → 42.5 (type: float)
Int 100 → 100.0 (type: float)
Float 3.14159 → 3.14159 (type: float)


## 2. Target Type Specification

Use `num_type` to specify whether the output should be int or float.

In [3]:
# Force integer conversion
int_result = to_num("42.9", num_type=int)
print(f"String '42.9' → int: {int_result} (type: {type(int_result).__name__})")

# Explicit float conversion
float_result = to_num("100", num_type=float)
print(f"String '100' → float: {float_result} (type: {type(float_result).__name__})")

# Note: int conversion truncates decimals
truncated = to_num(99.99, num_type=int)
print(f"Float 99.99 → int (truncated): {truncated}")

String '42.9' → int: 42 (type: int)
String '100' → float: 100.0 (type: float)
Float 99.99 → int (truncated): 99


## 3. Bounds Checking

Enforce valid ranges with `upper_bound` and `lower_bound`. Bounds are inclusive.

In [4]:
# Valid within bounds
result = to_num("50", lower_bound=0, upper_bound=100)
print(f"50 within [0, 100]: {result}")

# Boundary values are inclusive
min_val = to_num("0", lower_bound=0, upper_bound=100)
max_val = to_num("100", lower_bound=0, upper_bound=100)
print(f"Bounds are inclusive: min={min_val}, max={max_val}")

# Only lower bound
positive = to_num("42", lower_bound=0)
print(f"Positive number (lower_bound=0): {positive}")

# Only upper bound
capped = to_num("-10", upper_bound=0)
print(f"Negative number (upper_bound=0): {capped}")

50 within [0, 100]: 50.0
Bounds are inclusive: min=0.0, max=100.0
Positive number (lower_bound=0): 42.0
Negative number (upper_bound=0): -10.0


In [5]:
# Out of bounds raises ValueError
try:
    to_num("150", upper_bound=100)
except ValueError as e:
    print(f"✓ Upper bound violation: {e}")

try:
    to_num("-10", lower_bound=0)
except ValueError as e:
    print(f"✓ Lower bound violation: {e}")

✓ Upper bound violation: Value 150.0 exceeds upper bound 100
✓ Lower bound violation: Value -10.0 below lower bound 0


## 4. Precision Control

Round floats to specified decimal places with the `precision` parameter. Only applies when `num_type=float`.

In [6]:
# Round to 2 decimal places
price = to_num("19.996", precision=2)
print(f"Price with 2 decimals: ${price}")

# Round to 0 decimals (whole number float)
whole = to_num("42.7", precision=0)
print(f"Rounded to whole: {whole} (type: {type(whole).__name__})")

# High precision calculations
pi_approx = to_num("3.14159265359", precision=5)
print(f"Pi to 5 decimals: {pi_approx}")

# Precision with bounds
percent = to_num("0.12345", precision=2, lower_bound=0, upper_bound=1)
print(f"Percentage (2 decimals, [0,1]): {percent}")

Price with 2 decimals: $20.0
Rounded to whole: 43.0 (type: float)
Pi to 5 decimals: 3.14159
Percentage (2 decimals, [0,1]): 0.12


In [7]:
# Precision is ignored for integers
int_result = to_num("42.999", num_type=int, precision=2)
print(f"With num_type=int, precision ignored: {int_result}")

With num_type=int, precision ignored: 42


## 5. Special Input Types

Handle boolean, Decimal, and string edge cases.

In [8]:
# Boolean conversion (True=1.0, False=0.0)
true_val = to_num(True)
false_val = to_num(False)
print(f"Boolean: True → {true_val}, False → {false_val}")

# Decimal input (high precision)
decimal_input = Decimal("123.456789")
result = to_num(decimal_input, precision=2)
print(f"Decimal {decimal_input} → {result}")

# Whitespace is stripped from strings
spaced = to_num("  42.5  ")
print(f"String with whitespace '  42.5  ' → {spaced}")

# Negative numbers
negative = to_num("-273.15")
print(f"Negative string '-273.15' → {negative}")

Boolean: True → 1.0, False → 0.0
Decimal 123.456789 → 123.46
String with whitespace '  42.5  ' → 42.5
Negative string '-273.15' → -273.15


## 6. Error Handling

The function raises clear errors for invalid inputs.

In [9]:
# Empty string
try:
    to_num("")
except ValueError as e:
    print(f"Empty string: {e}")

# Whitespace-only string
try:
    to_num("   ")
except ValueError as e:
    print(f"Whitespace-only: {e}")

# Invalid string format
try:
    to_num("not_a_number")
except ValueError as e:
    print(f"Invalid format: {e}")

# Unsupported type
try:
    to_num([1, 2, 3])
except TypeError as e:
    print(f"Unsupported type: {e}")

# Invalid num_type
try:
    to_num("42", num_type=str)
except ValueError as e:
    print(f"Invalid num_type: {e}")

Empty string: Empty string cannot be converted to number
Whitespace-only: Empty string cannot be converted to number
Invalid format: Cannot convert 'not_a_number' to number
Unsupported type: Cannot convert list to number
Invalid num_type: Invalid number type: <class 'str'>


## 7. Real-World Examples

Practical use cases combining multiple features.

In [10]:
# Validate age input (integer, positive, reasonable bounds)
def validate_age(age_input):
    return to_num(age_input, num_type=int, lower_bound=0, upper_bound=120)


ages = ["25", "0", "120", 45]
for age_input in ages:
    age = validate_age(age_input)
    print(f"Valid age: {age}")

# Invalid age
try:
    validate_age("-5")
except ValueError as e:
    print(f"Invalid age: {e}")

Valid age: 25
Valid age: 0
Valid age: 120
Valid age: 45
Invalid age: Value -5.0 below lower bound 0


In [11]:
# Currency conversion with precision
def parse_price(price_str):
    return to_num(price_str, precision=2, lower_bound=0)


prices = ["19.99", "100", "0.99", "1234.567"]
for price in prices:
    result = parse_price(price)
    print(f"${price} → ${result:.2f}")

$19.99 → $19.99
$100 → $100.00
$0.99 → $0.99
$1234.567 → $1234.57


In [12]:
# Percentage validation (0-100 range, 1 decimal)
def validate_percentage(pct_input):
    return to_num(pct_input, precision=1, lower_bound=0, upper_bound=100)


percentages = ["0", "50.5", "100", "99.99"]
for pct in percentages:
    result = validate_percentage(pct)
    print(f"{pct}% → {result}%")

0% → 0.0%
50.5% → 50.5%
100% → 100.0%
99.99% → 100.0%


In [13]:
# Temperature conversion (Celsius, reasonable bounds)
def validate_celsius(temp_str):
    return to_num(temp_str, precision=1, lower_bound=-273.15, upper_bound=1000)


temps = ["-273.15", "0", "37.5", "100"]
for temp in temps:
    result = validate_celsius(temp)
    print(f"{temp}°C validated: {result}°C")

-273.15°C validated: -273.1°C
0°C validated: 0.0°C
37.5°C validated: 37.5°C
100°C validated: 100.0°C


## 8. Combining All Features

Complex validation scenarios using all parameters together.

In [14]:
# API rate limit: positive float, max 1000 requests/sec, 2 decimals
rate_limit = to_num(
    "123.456",
    num_type=float,
    lower_bound=0.01,
    upper_bound=1000.0,
    precision=2,
)
print(f"Rate limit: {rate_limit} req/sec")

# Probability: float in [0,1], high precision
probability = to_num(
    "0.12345678",
    num_type=float,
    lower_bound=0.0,
    upper_bound=1.0,
    precision=6,
)
print(f"Probability: {probability}")

# Grid coordinate: integer, bounded
x_coord = to_num(
    "42.9",
    num_type=int,
    lower_bound=-100,
    upper_bound=100,
)
print(f"Grid X coordinate: {x_coord}")

Rate limit: 123.46 req/sec
Probability: 0.123457
Grid X coordinate: 42


## Summary Checklist

**to_num() Essentials:**
- ✅ Converts strings, int, float, bool, Decimal to numeric types
- ✅ Target type control via `num_type` (int or float)
- ✅ Inclusive bounds validation with `upper_bound` and `lower_bound`
- ✅ Precision control for floats via `precision` parameter
- ✅ Automatic whitespace stripping for string inputs
- ✅ Clear error messages for invalid inputs and bound violations
- ✅ Boolean handling (True=1.0, False=0.0)
- ✅ Decimal support for high-precision inputs

**Common Patterns:**
- Age validation: `num_type=int, lower_bound=0, upper_bound=120`
- Currency: `precision=2, lower_bound=0`
- Percentage: `precision=1, lower_bound=0, upper_bound=100`
- Probability: `precision=6, lower_bound=0.0, upper_bound=1.0`

**Next Steps:**
- See other string handlers for text processing utilities
- Use in validation layers for API inputs
- Combine with Pydantic models for typed data structures